# Physical Optics tutorial

In this notebook we go through a full physical optics calculation of a simple Cassegrain telescope (the Submillimeter Array's 6m diameter, F/# 0.42 primary focal length antenna), with the calculations being carried out using `numpy` and `scipy` routines.

This serves as both an introduction to the Physical Optics technique used in PyPO, and as a test case for us to evaluate the PyPO algorithms.

In [2]:
import numpy as np
import scipy as sp

import PyPO

In normalized field units of $\sqrt{\mathrm{Watts}}$ for all fields, the field at a point $\mathbf{r}$ due to the induced currents on a scatterer are given by:

$$ \mathbf{E}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{J}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{M}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$

$$ \mathbf{H}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{M}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{J}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$


For propagation in freespace (or a lossless dielectric), these equations reduce to:

$$ \mathbf{E}(\mathbf{r}) =  \int_{S} \frac{k^2}{4\pi} e^{-ikR} \Bigg[ Z_m \Bigg[ \mathbf{J} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{J} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ - \mathbf{M}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$

$$ \mathbf{H}(\mathbf{r}) = \int_{S} \frac{k^2}{4\pi} e^{-ikR} \Bigg[ \frac{1}{Z_m} \Bigg[ \mathbf{M} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{M} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ + \mathbf{J}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$

where the integral is carried out over the surface of the source scatterer $S$ and:

* $ds$ is the area element of the source scatterer.
* $\mathbf{r}$ is the point at which the field is to be calculated.
* $\mathbf{r}_s$ is the point on the source scatterer.
* $\mathbf{R} = \mathbf{r} - \mathbf{r}_s$ is the vector distance from the source element to the target point.
* $R = |\mathbf{R}|$.
* $\mathbf{\hat{R}} = \mathbf{R} / R$ is the unit vector along $\mathbf{R}$.
* $k$ is the wavenumber in the medium.
* $Z_m = \sqrt{\mu / \epsilon}$ is the impedance of the medium.
* $\mathbf{J}_s$ is the electric current at the source scatterer.
* $\mathbf{M}_s$ is the magnetic current at the source scatterer.


The following code implements the integrand of each of the above surface integrals.

Where the normal of the source current element points away from the target point, the integrand returns zero. This implements the shading of the target by the source scatterer.

For scatterers with surface and bulk properties that allow for transmission through the scatterer, we would instead return the field contribution from the current element on the back side of the scatterer.

In [3]:
def E_field_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    R_vec = source_point - target_point
    R = np.sqrt(np.vecdot(R_vec, R_vec))
    R_hat = R_vec/R
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_R = np.vecdot(ns, R_hat)
    if norm_dot_R > 0:
        return np.zeros_like(js)
    
    kR = k * R
    kR_inv = 1/kR  # This is a handy optimization for both the look of the code and performance
    
    kR_inv_sum1 = -kR_inv**2 + 1j*kR_inv*(kR_inv**2 - 1)
    kR_inv_sum2 = 3*kR_inv**2 + 1j*kR_inv*(1 - 3*kR_inv**2)
    kR_inv_sum3 = kR_inv**2 + 1j*kR_inv
    
    js_dot_R = np.vecdot(R_hat, js)
    
    js_dot_R_R = R_hat*js_dot_R
    
    ms_cross_R = np.linalg.cross(R_hat, ms)
    
    green = k**2 / 4*np.pi * np.exp(-1j*kR) * As
    
    d_e_field = (Z0 * (js*kR_inv_sum1 + js_dot_R_R*kR_inv_sum2) - ms_cross_R*kR_inv_sum3) * green
    
    return d_e_field

    
def H_field_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    R_vec = source_point - target_point
    R = np.sqrt(np.vecdot(R_vec, R_vec))
    R_hat = R_vec/R
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_R = np.vecdot(ns, R_hat)
    if norm_dot_R > 0:
        return np.zeros_like(ms)
    
    kR = k * R
    kR_inv = 1/kR
    
    kR_inv_sum1 = -kR_inv**2 + 1j*kR_inv*(kR_inv**2 - 1)
    kR_inv_sum2 = 3*kR_inv**2 + 1j*kR_inv*(1 - 3*kR_inv**2)
    kR_inv_sum3 = kR_inv**2 + 1j*kR_inv
    
    ms_dot_R = np.vecdot(ms, R_hat)
    
    ms_dot_R_R = R_hat*ms_dot_R
    
    js_cross_R = np.linalg.cross(R_hat, js)
    
    green = k**2 / 4*np.pi * np.exp(-1j*kR) * As
    
    d_h_field = (1/Z0 * (ms*kR_inv_sum1 + ms_dot_R_R*kR_inv_sum2) + js_cross_R*kR_inv_sum3) * green
    
    return d_h_field

## Farfields

In the far field limit ($R\rightarrow\infty$), the above expressions reduce to:

$$\mathbf{E}(\mathbf{\hat{r}}) = \int_S \frac{-i k^2}{4\pi} e^{ik\mathbf{r}_s\cdot\mathbf{\hat{r}}} \Big[ Z_m\big(\mathbf{J} - (\mathbf{J}\cdot\mathbf{\hat{r}})\big) - \mathbf{\hat{r}}\times \mathbf{M}\Big] ds $$

$$\mathbf{H}(\mathbf{\hat{r}}) = \int_S \frac{-i k^2}{4\pi} e^{ik\mathbf{r}_s\cdot\mathbf{\hat{r}}} \Big[ \frac{1}{Z_m}\big(\mathbf{M} - (\mathbf{M}\cdot\mathbf{\hat{r}})\big) + \mathbf{\hat{r}}\times \mathbf{J}\Big] ds $$

where $\mathbf{\hat{r}}$ is the farfield direction vector.

In [ ]:
def E_farfield_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field. This should be a unit vector
                                                pointing in the farfield direction.
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    rprime = source_point
    r_hat = target_point
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_r_hat = np.vecdot(ns, r_hat)
    if norm_dot_r_hat > 0:
        return np.zeros_like(js)
    
    js_dot_r = np.vecdot(r_hat, js)
    
    js_dot_r_r = js_dot_r*r_hat
    
    r_cross_ms = np.linalg.cross(r_hat, ms)
    
    green = k**2 / 4*np.pi * np.exp(-1j*k*np.vecdot(rprime, r_hat)) * As
    
    d_e_field = (Z0 * (js - js_dot_r_r) - r_cross_ms) * green
    
    return d_e_field

    
def H_farfield_integrand(target_point, source_point, ns, As, js, ms, k, Z0):
    """Calculate the integrand for the surface integral over the source used in calculating
    the E field at a target_point.
    
    @param target_point np.ndarray(<float>)   The point at which to calculate the field
    @param source_point np.ndarray(<float>)   The center of the current element that contributes to the integral
    @param ns           np.ndarray(<float>)   The normal to the source surface at the center of the current element
    @param As           float                 The area of the source current element
    @param js           np.ndarray(<complex>) The electric current at the center of the current element
    @param ms           np.ndarray(<complex>) The magnetic current at the center of the current element
    @param k            float                 The wavenumber for the calculation in m^-1
    @param Z0           float                 The impedance of the propagation medium (377 Ohm for freespace)"""
    rprime = source_point
    r_hat = target_point
    
    # Check to see if the source current element points towards the target
    # This handles shading
    norm_dot_r_hat = np.vecdot(ns, r_hat)
    if norm_dot_r_hat > 0:
        return np.zeros_like(ms)
    
    ms_dot_r = np.vecdot(r_hat, ms)
    
    ms_dot_r_r = ms_dot_r*r_hat
    
    r_cross_js = np.linalg.cross(r_hat, js)
    
    green = k**2 / 4*np.pi * np.exp(-1j*k*np.vecdot(rprime, r_hat)) * As
    
    d_h_field = (Z0 * (ms - ms_dot_r_r) + r_cross_js) * green
    
    return d_h_field

## Evaluating the Surface Integral

Evaluation of the surface integral is the main computational challenge in performaing PO calculations.

For moderately large surfaces and non-paraxial angles between the source surface normal and the target point, the complex phase of the integrands oscilates rapidly due to the Green's function's $e^{-i k R}$ term.

The integral is performed using Gaussian quadrature in the $x$ and $y$ directions for rectangular grids and in the $\rho$ direction for polar grids.  For the $\phi$ direction of polar grids, where the two end points of the integration have the same value, the trapezoidal method is sufficiently accurate.

In the polar case, we integrate over the $\phi$ direction before the $\rho$ direction. This allows for the future use of lower numbers of PO points near the center of the scatterer.

Each evaluation of the integrands at a specific source point needs to know the value of the current element at that point.  Using a fully automatic integration routine like those in scipy.integrate to perform the integration would require that the field at each point on the source scatterer be evaluated by performing a similar integral over the source that is illuminating that scatterer.  This would require that the whole PO chain be calculated at once.

We can avoid this problem by dividing the integration routine into several steps: 
1. First designate the integration points on the source scatterer. This is equivalent to manually setting the number of points to be used.
2. Calculate the current elements at those points.
3. Calculate the fields at the target point(s).

This sequence can be repeated for each scatterer in the optics chain.  For the initial source, either an easily evaluated analytic function can be used to define the source currents, or an interpolation routine can be used to generate the currents at an arbitrary point on the source.

The designation of evaluation points on each scatterer requires that we use fixed order Gaussian quadrature, similar to that implemented by `scipy.integrate.fixed_quad`. For the trapezoid rule integration over $\phi$, we can use the `scipy.integrate.trapezoid` algorithm as is, supplying it with the precomputed values for the integrand and $\phi$.

We can break the `scipy.integrate.fixed_quad` routine into separate parts as follows:

In [11]:
def get_quad_pointsweights(n, a=-1.0, b=1.0):
    """Returns the points and weights for 1-d Gauss-Legendre integration of order n.
    
    @param int Number of points in the integral.
    @param float Lower limit of integration.
    @param float Upper limit of integration.
    
    @returns np.ndarrays x, w containing the x points and the weights"""
    if np.isinf(a) or np.isinf(b):
        raise ValueError("Gaussian quadrature is only available for finite limits")
    
    x, w = sp.integrate._quadrature._cached_roots_legendre(n)
    
    return (b-a)*(x+1)/2.0 + a, w

In [17]:
x, w = get_quad_pointsweights(201, 0, 3.0)

print("Evaluation points:\n", x)
print("Weights:\n", w)

Evaluation points:
 [1.06824846e-04 5.62825228e-04 1.38308857e-03 2.56759771e-03
 4.11608707e-03 6.02818557e-03 8.30343020e-03 1.09412686e-02
 1.39410600e-02 1.73020754e-02 2.10234978e-02 2.51044227e-02
 2.95438583e-02 3.43407254e-02 3.94938580e-02 4.50020036e-02
 5.08638233e-02 5.70778921e-02 6.36426997e-02 7.05566502e-02
 7.78180631e-02 8.54251733e-02 9.33761318e-02 1.01669006e-01
 1.10301779e-01 1.19272354e-01 1.28578550e-01 1.38218105e-01
 1.48188675e-01 1.58487837e-01 1.69113087e-01 1.80061843e-01
 1.91331444e-01 2.02919150e-01 2.14822144e-01 2.27037533e-01
 2.39562347e-01 2.52393543e-01 2.65528001e-01 2.78962529e-01
 2.92693861e-01 3.06718660e-01 3.21033515e-01 3.35634949e-01
 3.50519410e-01 3.65683282e-01 3.81122878e-01 3.96834446e-01
 4.12814166e-01 4.29058154e-01 4.45562462e-01 4.62323077e-01
 4.79335926e-01 4.96596873e-01 5.14101723e-01 5.31846220e-01
 5.49826052e-01 5.68036847e-01 5.86474180e-01 6.05133568e-01
 6.24010476e-01 6.43100315e-01 6.62398446e-01 6.81900177e-01
 7.0

We will calculate these grid points and weights when performing PO propagation to the source scatterer, and store them with the calculated PO currents on the scatterer.

We can now calculate the $\mathbf{J}_s$ and $\mathbf{M}_s$ at each of the points, and pass that to the final integrand, along with the source points, surface normals, element areas, etc..

In [ ]:
def fixed_quad_evaluate(func, x, args=None):
    """Evaluate function `func` at points in `x`.
    
    @param callable func    The function to evaluate.
    @param np.ndarray x     The points at which to evalute the function.
    @param tuple args       Extra arguments to pass to the function.
    """
    
    return func(x, *args)

The final integration is carried out with the following function:

In [ ]:
def fixed_quad_integrate(y, x, w, axis=0):
    """Compute a definite integral using fixed-order Gaussian quadrature at precalculated points.

    Integrate `y` over `x` using Gaussian quadrature with weights `w`.
    
    @param np.ndarray y     The integrand evaluated at points `x`.
    @param np.ndarray x     The points that the integrand was evaluated at.
    @param np.ndarray w     The weights to use in the integration.
    @param int axis         The axis of y to perform the sum over.
        
    @returns float   Gaussian quadrature approximation to the integral
    """
    a = x[0]
    b = x[-1]
    
    return (b-a)/2.0 * np.sum(w*y, axis=axis)

# Convergence of the integrals

The surface integrals converge fastest for focused reflectors at points close to the main lobe of the radiation pattern - in this region, the integrand only oscillates slowly in phase as the position across the source is varied.  As the observation point moves away from the main lobe into the far sidelobe regions, the phase of the integrand varies more rapidly, and a finer grid of points is required to adequately sample the integrands.

In general for the integral to converge, at least one sample per period of oscillation is required - the cycle of oscillation includes both the distance from source to the scattering, and from the scattering to the target point.  Thus we require a point spacing on the scatterer of $ \Delta r + \Delta r' \lt \lambda / 2 $.  For the worst case of illumination of a scatterer at an angle nearly parallel to the surface, and an observation point nearly anti-parallel to the incoming ray, this means that integration points on the scatterer must be spaced $ \lambda / 2 $ apart.  This gives an upper limit on the point spacings required.

For a reflector with a circular rim, the spacing between integration points on the rim is 

$$ \Delta r = \pi \frac{D}{n_\phi} $$

Thus the integral will converge if

$$ n_\phi \gt 2\pi \frac{D}{\lambda} $$

For a focused reflector with a circular rim of diameter $D$  observed at an angle $ \theta $ to the main beam, the integral will converge if:

$$ n_\rho \gt 2.4\Bigg(1.09\cdot\pi\frac{D}{\lambda}\sin\theta + 10 \Bigg) $$

and 

$$ n_\phi \gt 1.09\cdot\pi\frac{D}{\lambda}\sin\theta + 10 $$

For a focused reflector with a rectangular rim, the integral will converge for

$$ n_{x,y} = 1.75\cdot \frac{D_{x,y}}{\lambda}\sin \theta_{x,y} +10 $$

Again, the worst case is that a point spacing of 

$$ n_{x, y} = 3.5 \frac{D_{x,y}}{\lambda} $$